In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


In [48]:
#create a dataframe
labels_train = []
images_train = []

path_train = "D:\Coding\DataScience\DataPlayground\FruitRecognition\data\\train"
dir_list_train = os.listdir(path_train)

for lab in dir_list_train:
    #temp image list for each label
    img_list = os.listdir(path_train + "\\" + lab)
    for img in img_list:
        labels_train.append(lab)
        images_train.append(path_train + "\\" + lab + "\\" + img)

df_train = pd.DataFrame({"label":labels_train, "image":images_train})
#set each label a number
label_nums = dict(enumerate(df_train["label"].unique()))
#swap keys and values
label_nums = {value:key for key, value in label_nums.items()}

#map the label to the number
df_train["label"] = df_train["label"].map(label_nums)

#shuffle the dataframe
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train


,label,image
0,6,D:\Coding\DataScience\DataPlayground\FruitReco...
1,6,D:\Coding\DataScience\DataPlayground\FruitReco...
2,2,D:\Coding\DataScience\DataPlayground\FruitReco...
3,3,D:\Coding\DataScience\DataPlayground\FruitReco...
4,2,D:\Coding\DataScience\DataPlayground\FruitReco...
...,...,...
2296,2,D:\Coding\DataScience\DataPlayground\FruitReco...
2297,3,D:\Coding\DataScience\DataPlayground\FruitReco...
2298,3,D:\Coding\DataScience\DataPlayground\FruitReco...
2299,3,D:\Coding\DataScience\DataPlayground\FruitReco...


In [49]:
labels_test = []
images_test = []

path_test = "D:\Coding\DataScience\DataPlayground\FruitRecognition\data\\test"
dir_list_test = os.listdir(path_test)

for lab in dir_list_test:
    #temp image list for each label
    img_list = os.listdir(path_test + "\\" + lab)
    for img in img_list:
        labels_test.append(lab)
        images_test.append(path_test + "\\" + lab + "\\" + img)

df_test = pd.DataFrame({"label":labels_test, "image":images_test})
#set each label a number
label_nums = dict(enumerate(df_test["label"].unique()))
#swap keys and values
label_nums = {value:key for key, value in label_nums.items()}

#map the label to the number
df_test["label"] = df_test["label"].map(label_nums)

#shuffle the dataframe
df_test = df_test.sample(frac=1).reset_index(drop=True)

df_test


,label,image
0,9,D:\Coding\DataScience\DataPlayground\FruitReco...
1,6,D:\Coding\DataScience\DataPlayground\FruitReco...
2,0,D:\Coding\DataScience\DataPlayground\FruitReco...
3,0,D:\Coding\DataScience\DataPlayground\FruitReco...
4,9,D:\Coding\DataScience\DataPlayground\FruitReco...
...,...,...
1020,5,D:\Coding\DataScience\DataPlayground\FruitReco...
1021,8,D:\Coding\DataScience\DataPlayground\FruitReco...
1022,6,D:\Coding\DataScience\DataPlayground\FruitReco...
1023,9,D:\Coding\DataScience\DataPlayground\FruitReco...


In [50]:
#prediction dataframe
images_predict = []

path_predict = "D:\Coding\DataScience\DataPlayground\FruitRecognition\data\\predict"
img_list_predict = os.listdir(path_predict)

for img in img_list_predict:
    images_predict.append(path_predict + "\\" + img)

df_predict = pd.DataFrame({"image":images_predict})


In [51]:
#function to convert image to array of shape (100,100,3)
from PIL import Image
def img_to_array(img_path):
    img = Image.open(img_path)
    img = img.resize((28,28))
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = np.array(img)
    img_array = np.array(img)
    img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
    return img


In [52]:
#convert images to arrays
df_train["image"] = df_train["image"].apply(img_to_array)
df_test["image"] = df_test["image"].apply(img_to_array)

In [65]:

X_train = np.stack(df_train["image"].values)
X_test = np.stack(df_test["image"].values)
image_shape = (28,28, 3)

import tensorflow as tf
#train model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=image_shape),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, np.array(df_train["label"].tolist()), epochs=10, batch_size=64)

#test model
model.evaluate(X_test, np.array(df_test["label"].tolist()))

Epoch 1/10
36/36 [==============================] - 7s 129ms/step - loss: 11.4951 - accuracy: 0.1304
Epoch 2/10
36/36 [==============================] - 5s 132ms/step - loss: 2.1543 - accuracy: 0.1886
Epoch 3/10
36/36 [==============================] - 5s 132ms/step - loss: 2.0068 - accuracy: 0.2547
Epoch 4/10
36/36 [==============================] - 5s 127ms/step - loss: 1.8893 - accuracy: 0.3159
Epoch 5/10
36/36 [==============================] - 5s 132ms/step - loss: 1.6563 - accuracy: 0.4033
Epoch 6/10
36/36 [==============================] - 5s 131ms/step - loss: 1.3885 - accuracy: 0.4885
Epoch 7/10
36/36 [==============================] - 5s 129ms/step - loss: 1.2174 - accuracy: 0.5563
Epoch 8/10
36/36 [==============================] - 5s 131ms/step - loss: 1.0586 - accuracy: 0.6219
Epoch 9/10
36/36 [==============================] - 5s 133ms/step - loss: 0.9487 - accuracy: 0.6619
Epoch 10/10
33/33 [==============================] - 1s 14ms/step - loss: 1.6872 - accuracy: 0.4693

[1.6872484683990479, 0.4692682921886444]